In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### <b>식물 이미지 분류 학습</b>

* 순서
    1. 이미지 크롤링(image crawling) 혹은 크롤링 라이브러리
    2. 전이 학습(transfer learning)
    3. 웹 API 개발
* <b>식물</b> 분류기(classifier)
* 과정
    * 데이터 수집/정제
    * 인공지능 모델 학습
    * 학습된 모델 배포
* 한글 폰트 설치 이후에 <b>[런타임] - [런타임 다시 시작]</b>을 수행

In [ ]:
# 한글 폰트 설치하기 (꼭! 설치가 완료되면 [런타임 다시 시작]을 누르고 다시 실행하기)
!apt install fonts-nanum -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정하기
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,743 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [ ]:
# 필요한 라이브러리 설치하기
!git clone https://github.com/ndb796/bing_image_downloader

Cloning into 'bing_image_downloader'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 86 (delta 38), reused 46 (delta 16), pack-reused 0
Unpacking objects: 100% (86/86), done.


#### <b>1. 이미지 크롤링을 활용한 학습 이미지 수집</b>

* 수집한 이미지를 저장하기 위한 폴더를 생성하고, 필요한 함수를 정의합니다.

In [ ]:
import os
import shutil
from bing_image_downloader.bing_image_downloader import downloader


directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

# 초기 디렉토리 만들기
for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# 수집한 이미지를 학습 데이터와 평가 데이터로 구분하는 함수
def dataset_split(query, train_cnt):
    # 학습 및 평가 데이터셋 디렉토리 만들기
    for directory in directory_list:
        if not os.path.isdir(directory + '/' + query):
            os.makedirs(directory + '/' + query)
    # 학습 및 평가 데이터셋 준비하기
    cnt = 0
    for file_name in os.listdir(query):
        if cnt < train_cnt:
            print(f'[Train Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/train/' + query + '/' + file_name)
        else:
            print(f'[Test Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/test/' + query + '/' + file_name)
        cnt += 1
    shutil.rmtree(query)

* <b>식물</b> 이미지 크롤링을 진행하고 데이터셋을 구축합니다.

In [ ]:
arrays=[
        'Gaultheria procumbens',
'Dracaena sanderiana Virens',
'Cupressus macrocarpa Wilma Goldcrest',
'Caryota mitis',
'Begonia  spp',
'Rhapis  excelsa',
'Pelargonium rosium',
'Guzmania lingulata',
'Clivia miniata',
'Glecoma hederacea Variegata',
'Osmanthus fragrans',
'Euonymus japonica',
'Aucuba japonica  var. variegata',
'Zamioculcas zamiifolia',
'Dracaena sanderiana',
'Gynura aurantiaca',
'Begonia semperflorens',
'Aerides japonica',
'Podocarpus macrophyllus',
'Nandina domestica',
'Nematanthus gregarius',
'Neoregelia carolinae',
'Senecio rowleyanus',
'Impatiens hybrid New Guinea',
'Ficus retusa',
'Nephrolepis cordifolia Duffii',
'Beaucarnea recurvata (Nolina tuberculata)',
'Ficus elastica var. decora',
'Dendrobium phalaenopsis',
'Cyrtomium falcatum',
'Pittosporum tobira',
'Camellia japonica',
'Polyscias balfouriana',
'Duranta reptans',
'Dracaena reflexa Song of Jamaica',
'Dracaena deremensis Warneckii',
'Dracaena reflexa Song of India',
'Dracaena angustifolia Java',
'Dracaena deremensis Virens Compacta',
'Dracaena concinna Tricolor Rainbow',
'Dracaena draco',
'Dracaena marginata',
'Dracaena fragrans Massangeana',
'Dracaena sanderiana',
'Dracaena sanderiana Celes',
'Dracaena surculosa (D. godseffiana)',
'Dizygotheca elegantissima',
'Dieffenbachia Marianne',
'Dieffenbachia amoena Tropic snow',
'Ficus lyrata',
'Ceropegia woodii',
'Begonia rex',
'Rumohra adiantiformis',
'Ruscus spp',
'Maranta leuconeura',
'Trachelospermum asiaticum',
'Rohdea japonica',
'Mandevilla sanderi',
'Zamia pumila',
'Ficus elastica Robusta',
'Begonia lucerna',
'Monstera deliciosa',
'Rhapis excelsa Variegata',
'Trachelospermum asiaticum var.variegatum',
'Ficus benjamina Variegata',
'Ardisia pusilla Variegata',
'Acorus gramineus var. variegatus',
'Schefflera arboricola Hong Kong Variegata',
'Alpinia zerumbet Variegata',
'Yucca elephantipes Variegata',
'Chlorophytum comosum var. variegatum',
'Farfugium japonicum Argenteum',
'Fatshedera lizei Variegata',
'Ficus pumila Variegata',
'Muehlenbeckia complexa',
'Saxifraga stolonifera',
'Platycerium  bifurcatum',
'Farfugium japonicum',
'Ardisia crenata',
'Serissa foetida',
'Trachelospermum asiaticum var. majus',
'Ficus benjamina',
'Ficus benjamina King',
'Ficus benghalensis',
'Callistemon spp',
'Nephrolepis exaltata Bostoniensis',
'Bougainvillea glabra',
'Brunfelsia australis Bench',
'Phlebodium aureum',
'Vriesea',
'Chlorophytum bichetii',
'Chlorophytum comosum var. vittatum',
'Sansevieria trifasciata',
'Sansevieria trifasciata Golden Hahnii',
'Ardisia pusilla',
'Decora elastica Decora Tricolor',
'Humata tyermannii (Davallia griffithiana)',
'Calanthe discolor',
'Acorus gramineus',
'Senecio radicans',
'Chamaedorea seifrizii Burret',
'Selaginella',
'Cycas revoluta',
'Ficus elastica Sofia',
'Soleirolia soleirolii',
'Peperomia sandersii',
'Pilea cadierei',
'Tillandsia usneoides',
'Iberis sempervirens',
'Schefflera arboricola',
'Epipremnum aureum',
'Sansevieria cylindrica',
'Spathiphyllum wallisii',
'Spathiphyllum Clevelandii',
'Cissus antarctica',
'Cyclamen persicum',
'Cymbidium spp.',
'Syngonium podophyllum',
'Aglaonema commutatum',
'Adenium obesum',
'Adiantum raddianum',
'Araucaria heterophylla',
'Dypsis lutescens',
'Hippeastrum hybridum',
'Asplenium nidus',
'Viburnum odoratissimum var. awabuki',
'Hedera helix',
'Aphelandra squarrosa',
'Saintpaulia spp.',
'Anthurium andraeanum',
'Alocasia amazonica',
'Alocasia cucullata',
'Zebrina pendula',
'Peperomia caperata',
'Aechmea fasciata',
'Begonia X hiemalis',
'Acalypha reptans',
'Aspidistra elatior',
'Oxalis triangularis',
'Oncidium spp.',
'Ophiopogon japonicus',
'Rhapis multifida',
'Ficus umbellata',
'Yucca',
'Euonymus japonica Albomarginata',
'Ixora chinensis',
'Ficus microcarpa Ginseng',
'Ardisia japonica',
'Bletilla striata',
'Rhoeo discolor',
'Chlorophytum comosum',
'Pelargonium hortorum',
'Cyperus alternifolius',
'Rhapis humilis',
'Podocarous nagi',
'Peperomia puteolata',
'Gardenia jasminoides',
'Calathea makoyana',
'Calathea insignis',
'Calathea crocata',
'Caladium spp.',
'Kalanchoe blossfeldiana',
'Coffea arabica',
'Howea forsteriana',
'Cordyline terminalis Rededge',
'Codiaeum variegatum',
'Codiaeum variegatum var. pictum',
'Cryptanthus spp.',
'Clematis florida',
'Tradescantia sillamontana',
'Farfugium japonicum',
'Chamaedorea elegans',
'Tolmiea menziesii',
'Tillandsia cyanea',
'Pachira aquatica',
'Paphiopedilum hybrids',
'Phalaenopsis spp.',
'Fatsia japonica',
'Ficus panda',
'Fatshedera lizei',
'Peperomia obtusifolia',
'Peperomia clusiifolia',
'Peperomia puteolata',
'Euphorbia pulcherrima',
'Polyscias balfouriana Marginata',
'Ficus pumila',
'Pteris cretica',
'Pyracantha coccinea',
'Fittonia  verschaffeltii Pink Star',
'Fittonia  verschaffelti White Star',
'Chlorophytum comosum Picturatum',
'Pilea grauca',
'Philodendron Sunlight',
'Philodendron Xanadu',
'Philodendron goeldii',
'Philodendron Lemon Lime',
'Philodendron selloum',
'Philodendron oxycardium',
'Philodendron Congo',
'Haemaria discolor var. dawsoniana',
'Hemionitis arifolia',
'Heteropanax fragrans',
'Nerium indicum',
'Epipremnum aureum',
'Epipremnum aureum Lime',
'Hoya carnosa',
'Hoya carnosa Exotica Hort',
'Leptospermum scoparium',
'Peperomia angula',
'Trachelospermum asiaticum var. variegatum',
'Dracaena sanderiana',
'Ternstroemia japonica',
'Zephyranthes candida',
'Tradescantia albiflora Kunth Albovittata',
'Lavandula',
'Mentha × piperita',
'Plectranthus hadiensis var. tomentosus',
'Asparagus pulmosus',
'Cyrtomium caryotideum',
'Hypoestes phyllostachya'
]
for query in arrays:
  downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
  dataset_split(query, 30)

#### <b>2. PyTorch를 이용한 전이 학습(Transfer Learning)</b>

* 학습을 위해 필요한 라이브러리를 불러옵니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

* 데이터셋을 불러옵니다.

In [ ]:
# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

* 간단히 이미지를 시각화해 봅시다.

In [ ]:
def imshow(input, title):
    # torch.Tensor를 numpy 객체로 변환
    input = input.numpy().transpose((1, 2, 0))
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    plt.title(title)
    plt.show()


# 학습 데이터를 배치 단위로 불러오기
iterator = iter(train_dataloader)

# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화
inputs, classes = next(iterator)
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

* 학습할 CNN 딥러닝 모델 객체를 초기화합니다.

In [ ]:
model = models.resnet34(pretrained=True)
num_features = model.fc.in_features
# 전이 학습(transfer learning): 모델의 출력 뉴런 수를 3개로 교체하여 마지막 레이어 다시 학습
model.fc = nn.Linear(num_features, 3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

* 학습을 진행합니다.

In [ ]:
num_epochs = 50
model.train()
start_time = time.time()

# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epochs):
    running_loss = 0.
    running_corrects = 0

    # 배치 단위로 학습 데이터 불러오기
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.

    # 학습 과정 중에 결과 출력
    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

* 학습된 모델을 평가합니다.

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
        print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch_loss = running_loss / len(test_datasets)
    epoch_acc = running_corrects / len(test_datasets) * 100.
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))

#### <b>3. 분류 모델 API 개발</b>

* 학습된 분류 모델을 다른 사람이 사용할 수 있도록 API를 개발하여 배포합니다.
* 먼저 한 장의 이미지를 파일로부터 읽어와 분류 결과를 반환하는 기능을 작성합니다.

In [ ]:
# 테스트용 이미지 다운로드하기
!wget https://upload.wikimedia.org/wikipedia/commons/f/f1/Don_Lee_by_Gage_Skidmore.jpg -O test_image.jpg

In [ ]:
from PIL import Image


image = Image.open('test_image.jpg')
image = transforms_test(image).unsqueeze(0).to(device)

with torch.no_grad():
    outputs = model(image)
    _, preds = torch.max(outputs, 1)
    imshow(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

* 웹 API 개방을 위해 <b>Ngrok</b> 서비스를 이용합니다.
* API 기능 제공을 위해 <b>Flask 프레임워크</b>를 사용합니다.

In [ ]:
# 필요한 라이브러리 설치하기
!pip install flask-ngrok

In [ ]:
import io
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request


# 이미지를 읽어 결과를 반환하는 함수
def get_prediction(image_bytes):
    image = Image.open(io.BytesIO(image_bytes))
    image = transforms_test(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
        imshow(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    return class_names[preds[0]]


app = Flask(__name__)


@app.route('/', methods=['POST'])
def predict():
    if request.method == 'POST':
        # 이미지 바이트 데이터 받아오기
        file = request.files['file']
        image_bytes = file.read()

        # 분류 결과 확인 및 클라이언트에게 결과 반환
        class_name = get_prediction(image_bytes=image_bytes)
        print("결과:", {'class_name': class_name})
        return jsonify({'class_name': class_name})

* API를 개방할 수 있으며 실행할 때마다 서버의 주소가 변경됩니다.
* 서버 주소를 정확히 확인할 필요가 있습니다.

In [ ]:
run_with_ngrok(app)
app.run()

* 클라이언트 테스트 방법

<pre>
curl -X POST -F file=@{이미지 파일명} {Ngrok 서버 주소}
</pre>

* 사용 예시

<pre>
curl -X POST -F file=@dongseok.jpg http://c4cdb8de3a35.ngrok.io/
</pre>

* 이러한 방식으로 웹, 모바일, 게임 등의 소프트웨어 환경에서 API를 호출하는 방식으로 인공지능 앱을 개발할 수 있습니다.